In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# 1 - Develop CNN and Resnet50

## Data processing

In [3]:
# Training data generator with augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

# Test data generator (no augmentation, just rescale)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Load the testing dataset
test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Don't shuffle so the images are in the same order for evaluation
)

Found 750 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## Model definition

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Simple CNN Model
simple_model = Sequential()

# 1st Convolutional Block
simple_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
simple_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the convolutional outputs into a 1D vector
simple_model.add(Flatten())

# Dense layer with 10 neurons
simple_model.add(Dense(10, activation='relu'))

# Output layer with Softmax activation for binary classification
simple_model.add(Dense(2, activation='softmax'))

# Compile the simple model
simple_model.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Print model summary
simple_model.summary()


c:\Users\quang\src\uni\2024-sem2\cos40007\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 175232)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │     1,752,330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,753,248 (6.69 MB)

 Trainable params: 1,753,248 (6.69 MB)

 Non-trainable params: 0 (0.00 B)

## Model training

In [5]:
# Train the simple model
history_simple = simple_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/10


c:\Users\quang\src\uni\2024-sem2\cos40007\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 248ms/step - accuracy: 0.5267 - loss: 1.6684 - val_accuracy: 0.8000 - val_loss: 0.5223
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7188 - loss: 0.6030 - val_accuracy: 0.8000 - val_loss: 0.5183
Epoch 3/10


C:\Users\quang\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 240ms/step - accuracy: 0.7256 - loss: 0.5725 - val_accuracy: 0.8000 - val_loss: 0.4741
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8438 - loss: 0.4922 - val_accuracy: 0.8000 - val_loss: 0.4662
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 250ms/step - accuracy: 0.7666 - loss: 0.5201 - val_accuracy: 0.8000 - val_loss: 0.4264
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8125 - loss: 0.5070 - val_accuracy: 0.9000 - val_loss: 0.4236
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 244ms/step - accuracy: 0.7979 - loss: 0.4912 - val_accuracy: 0.9000 - val_loss: 0.4266
Epoch 8/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.8438 - loss: 0.3881

## Evaluate on test set

In [8]:
simple_loss, simple_accuracy = simple_model.evaluate(test_generator)
print(f"Simple Model Accuracy: {simple_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.9000 - loss: 0.3287
Test accuracy: 90.00%


## Make Predictions on Test Images

In [9]:
import numpy as np
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)  # Classify as 1 (rust) or 0 (no-rust)

# Compare with actual labels
true_classes = test_generator.classes
accuracy_per_image = np.sum(predicted_classes == true_classes) / len(true_classes)
print(f"Accuracy on individual test images: {accuracy_per_image * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Accuracy on individual test images: 1000.00%
